
<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Image source: https://www.kaggle.com/
</p>

## Stage 1: Installing dependencies and setting up GPU environment

In [1]:
!pip install tensorflow-gpu==2.0.0.alpha0

ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0.0.alpha0 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0rc0, 2.7.0rc1, 2.7.0)
ERROR: No matching distribution found for tensorflow-gpu==2.0.0.alpha0


## Stage 2: Import dependencies for the project

In [2]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [3]:
tf.__version__

'2.7.0'

## Stage 3: Dataset preprocessing



### Loading the dataset

In [4]:
#Loading the Fashion Mnist dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


### Image normalization

We devide each image in the training and testing dataset with the maxiumum number of pixels (255).

In this way each pixel will be in the rainge [0, 1]. By normalizing imaes we are making sure that our model (ANN) trains faster.

In [5]:
X_train = X_train / 255.0

In [6]:
X_test = X_test / 255.0

### Reshaping of the dataset

Since we are using fully connected network, we reshape the training and testing subsets to be in the vector format. 

In [7]:
#Since each image is 28x28, we simply use reshape the full dataset to [-1 (all elements), height * width]
X_train = X_train.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

In [9]:
#Reshape the testing subset in the same way
X_test = X_test.reshape(-1, 28*28)

## Stage 4: Building an Artificial Neural network

### Defining the model

Simply define an object of the Sequential model.

In [10]:
model = tf.keras.models.Sequential()

### Adding the first layer (Dense layer)

Layer hyper-parameters:
- number of units/neurons: 128
- activation function: ReLU
- input_shape: (784, )

In [11]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))

### Adding a Dropout layer 

Dropout is a Regularization technique where we randomly set neurons in a layer to zero. In this way, while training those neurons won't be updated. Because some percentage of neurons won't be updated the whole training process is long and we have less chance for overfitting.

In [12]:
model.add(tf.keras.layers.Dropout(0.2))

##Adding Second layer 

In [13]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(128, )))

### Adding a Third layer 

In [14]:
model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(128, )))

### Adding the fourth layer (output layer)

- units == number of classes (10 in the case of Fashion MNIST)
- activation = 'softmax'

In [15]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Comiling the model

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [16]:
opt = tf.keras.optimizers.Adam(learning_rate =0.001)
model.compile(optimizer = opt, loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 125,898
Trainable params: 125,898
Non-trainable params: 0
_________________________________________________________________


### Training the model

In [18]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 3ms/step - loss: 0.5404 - sparse_categorical_accuracy: 0.8035
Epoch 2/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.4108 - sparse_categorical_accuracy: 0.8493
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3751 - sparse_categorical_accuracy: 0.8621
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3524 - sparse_categorical_accuracy: 0.8687
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3393 - sparse_categorical_accuracy: 0.8743
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3269 - sparse_categorical_accuracy: 0.8785
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3163 - sparse_categorical_accuracy: 0.8828
Epoch 8/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3081 - sparse_categorical_accuracy: 0.8856
Epoch 9/10
1875/1875 [==========

### Model evaluation and prediction

In [19]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3338 - sparse_categorical_accuracy: 0.8809


In [20]:
print("Test accuracy: {}".format(test_accuracy))

Test accuracy: 0.8809000253677368


## Stage 5 : Saving the model

### Saving the architecture (topology) of the network

In [21]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Saving network weights

In [22]:
model.save_weights("fashion_model.h5")